# Select Model

In [21]:
models = {
    "gpt-4o": "gpt-4o",
    "DeepSeek-R1": "DeepSeek-R1",
    "Phi-4-mini-instruct": "Phi-4-mini-instruct",
    "Phi-4-multimodal-instruct (version:1)": "Phi-4-multimodal-instruct",

    "o3-mini": "o3-mini",
}

# Display available models
print("Available models:")
for i, model_name in enumerate(models.keys()):
    print(f"{i+1}. {model_name}")

# Prompt user for selection
selection = input("Select a model (enter number): ")
try:
    selection_idx = int(selection) - 1
    model_keys = list(models.keys())
    if 0 <= selection_idx < len(model_keys):
        selected_model = models[model_keys[selection_idx]]
        print(f"Selected model: {selected_model}")
    else:
        print("Invalid selection. Using default model 'gpt-4o'")
        selected_model = "gpt-4o"
except ValueError:
    print("Invalid input. Using default model 'gpt-4o'")
    selected_model = "gpt-4o"

Available models:
1. gpt-4o
2. DeepSeek-R1
3. Phi-4-mini-instruct
4. Phi-4-multimodal-instruct (version:1)
5. o3-mini
Selected model: DeepSeek-R1


# Loading Libraries

In [22]:
import os
from openai import OpenAI


In [23]:
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

from azure.identity import DefaultAzureCredential, get_bearer_token_provider, AzureCliCredential



In [24]:
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    ToolMessage,
    AssistantMessage
)

# Initilise a Chat Completion Client

In [25]:
endpoint = os.getenv("AZURE_INFERENCE_SDK_ENDPOINT")

In [26]:
client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(os.getenv("AZURE_INFERENCE_SDK_ENDPOINT_KEY")),
    credential_scopes=["https://cognitiveservices.azure.com/.default"],

    )

## Run a basic Chat Completion test

In [27]:
messages = [
    UserMessage(content="1 shirt needs 1 hour to dry, how long does it take to dry 3 shirts?"),
]

In [28]:
print(messages[0].content)
# gpt-4o
response = client.complete(
    messages = messages,
    model = selected_model
)
print(response.choices[0].message.content)

1 shirt needs 1 hour to dry, how long does it take to dry 3 shirts?
<think>
Okay, so the question is: "If 1 shirt takes 1 hour to dry, how long does it take to dry 3 shirts?" At first glance, it seems straightforward, but let me think about it carefully.

Hmm, when they say 1 shirt needs 1 hour to dry, is that assuming that you're drying just one shirt by itself? Like, if you hang one shirt outside, it dries in an hour. Now, if you have three shirts, do you need more time, or is it still the same? Wait, maybe it depends on how you're drying them. If you have enough space and the drying conditions are the same for each shirt, then maybe all three can dry simultaneously without affecting each other's drying time. So, if you put three shirts on a clothesline, each exposed to the same amount of air and sunlight, each would take 1 hour to dry, right? So, regardless of the number, as long as they're dried at the same time, it's still 1 hour.

But maybe the question is trying to trick me. Som

# Tool Calling

![Tool calling flow](https://techcommunity.microsoft.com/t5/s/gxcuf89792/images/bS00MzkxMDI5LUM2YTczNA?revision=7&image-dimensions=2000x2000&constrain-image=true "Tool calling flow")


Using tools requires the following parts 
- Tool definition using `ChatCompletionsToolDefinition` and `FunctionDefinition`
- Tool function, e.g. `get_weather`
- Tool message holding the result of a tool call, using `ToolMessage`

The process to perform a tool call: 
1. Define the tool using `ChatCompletionsToolDefinition` and create a function 

2. Create a chat completion request and pass an array of defined tools 
   1. append response message to chat history (`messages`)

3. Inspect the response from the chat completion for existence of tool calls, `response.choices[0].message.tool_calls`
   1. In case a tool call existes in response, extract the tool call details (function name, arguments, tool call id) and call the function
   2. Create a `ToolMessage` object with tool call id and function result as content
   3. Append the `ToolMessage` to the chat history (`messages`)

4. Send the updated chat history to the chat completion API again

references:
- https://learn.microsoft.com/en-us/azure/ai-foundry/model-inference/how-to/use-chat-completions?pivots=programming-language-python#use-tools

## Essential lirbaries for Tool Calling

In [29]:
import json
import logging

from azure.ai.inference.models import (
    FunctionDefinition,
    ChatCompletionsToolDefinition,

    SystemMessage,
    UserMessage,
    ToolMessage,

    CompletionsFinishReason,

)

from azure.ai.inference import (
    ChatCompletionsClient
)

## Tool Functions

This section shows the function definition and the function decleration itself.


This schema represents the function signature and the parameters of the function. The function itself is defined in the next section.

```python
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name, e.g. San Francisco",
                    },
                },
                "required": ["location"],
            },
        }
    }
]
```

Tools will be defined using SDK classes

In [30]:
func_def_get_weather = FunctionDefinition(
    name="get_weather",
    description="Get the weather for a given city",
    parameters={
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to get the weather for"
            },
            "unit": {
                "type": "string",
                "description": "The unit to use for the temperature (C or F)"
            }
        },
        "required": ["city"]
    }
)

func_def_get_current_time = FunctionDefinition(
    name="get_current_time",
    description="Get the current time in a given location",
    parameters={
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city name, e.g. San Francisco"
            }
        },
        "required": ["location"]
    }
)


tools = [
    ChatCompletionsToolDefinition(function=func_def_get_weather),
    ChatCompletionsToolDefinition(function=func_def_get_current_time)
]

In [31]:
def get_weather(city, unit="C"):
    """Get the weather for a given city"""
    print(f"get_weather called with city: {city}, unit: {unit}")
    # Simulate getting weather data
    weather_data = {
        "city": city,
        "temperature": "20",
        "unit": unit,
        "description": "Sunny"
    }
    return json.dumps(weather_data)

def get_current_time(location):
    from datetime import datetime
    """Get the current time for a given location"""
    print(f"get_current_time called with location: {location}")
    location_lower = location.lower()


    current_time = datetime.now().strftime("%I:%M %p")
    return json.dumps({
        "location": location,
        "current_time": current_time
    })

    print(f"No timezone data found for {location_lower}")
    return json.dumps({"location": location, "current_time": "unknown"})

print(get_weather("Sydney", "C"))
print(get_current_time("Sydney"))

get_weather called with city: Sydney, unit: C
{"city": "Sydney", "temperature": "20", "unit": "C", "description": "Sunny"}
get_current_time called with location: Sydney
{"location": "Sydney", "current_time": "05:22 PM"}


## The first call 

In the first call we pass th user message in the message history, and the tool definition in the tools parameter.

Must inspect the finish reason in the response. If the finish reason is `TOOL_CALLS` then we need to call the function and pass the result back to the chat completion API.

In [32]:
message_history = [
    SystemMessage(content="You are a helpful assistant."),
    UserMessage(content="What is the time and weather in Sydney?")
]


first_response = client.complete(
    messages=message_history,
    model = selected_model,
    tools=tools,
    tool_choice="auto"
)

#inspect first_response
print("finish reason: ", first_response.choices[0].finish_reason)
first_response.choices[0].message.as_dict()

finish reason:  CompletionsFinishReason.STOPPED


{'content': "<think>\nOkay, the user is asking for the current time and weather in Sydney. Since I can't access real-time data, I need to let them know that. But I should still be helpful. Maybe suggest checking a reliable website or app for that info. Also, mention that Sydney is in Australia, just in case there's any confusion with other Sydneys. Keep the response friendly and offer further help if needed.\n\nAlright, structure the response: Start by stating I can't provide real-time data. Then suggest using a weather website or app. Mention the time zone part. Offer to help convert time zones if they need. Keep it concise and polite.\n</think>\n\nI can't provide real-time data like current weather or time, but you can quickly check these details for Sydney, Australia, using services like **Google Search**, **Weather.com**, or a weather app. Sydney typically follows Australian Eastern Daylight Time (AEDT) UTC+11 during daylight saving (Oct-Apr) and Australian Eastern Standard Time (A

### Finish reasons

`CompletionsFinishReason` is an enum that defines the possible reasons for a completion to finish.


The enum values are:
```python
CompletionsFinishReason.CONTENT_FILTERED
CompletionsFinishReason.STOPPED
CompletionsFinishReason.TOOL_CALLS
CompletionsFinishReason.TOKEN_LIMIT_REACHED
```

In [37]:
finish_reason = first_response.choices[0].finish_reason

response_message = first_response.choices[0].message
tool_calls = response_message.tool_calls if finish_reason == CompletionsFinishReason.TOOL_CALLS else []

## Perform Tool Calls

In [34]:
# append response_message to chat history and perform tool calls
message_history.append(response_message)

In [35]:
message_history

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'What is the time and weather in Sydney?'},
 {'content': "<think>\nOkay, the user is asking for the current time and weather in Sydney. Since I can't access real-time data, I need to let them know that. But I should still be helpful. Maybe suggest checking a reliable website or app for that info. Also, mention that Sydney is in Australia, just in case there's any confusion with other Sydneys. Keep the response friendly and offer further help if needed.\n\nAlright, structure the response: Start by stating I can't provide real-time data. Then suggest using a weather website or app. Mention the time zone part. Offer to help convert time zones if they need. Keep it concise and polite.\n</think>\n\nI can't provide real-time data like current weather or time, but you can quickly check these details for Sydney, Australia, using services like **Google Search**, **Weather.com**, or a weather app. Sydney

### Executing function calls using locals() and dictionary unpacking with ** 


All the function calls are equivalent to each other. 


```python
function_name= "get_weather"
function_args = {
    "city": "Sydney",
    "unit": "C"
}

locals()[function_name](**function_args)
# equivalant to
locals()['get_weather'](**{'city': 'Sydney', 'unit': 'C'})\
# and
get_weather(city="Sydney", unit="C")
# and
get_weather(**{"city": "Sydney", "unit": "C"})
```

and will produce the same output

In [38]:
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    tool_call_id = tool_call.id



    # execute the function call and append the results as a ToolMessage in the message history
    function_call_results = locals()[function_name](**function_args)

    message_history.append(
        ToolMessage(
            tool_call_id=tool_call_id,
            content=function_call_results
        )
    )


    print(f"Tool Call ID: {tool_call_id}")
    print(f"Function: {function_name}")
    print(f"Arguments: {json.dumps(function_args, indent=2)}")
    print(f"Results: {function_call_results}")
    print("---")



In [39]:
## inspect message history after adding tool call results
for message in message_history:
    print(message.as_dict())
    print("---")

{'role': 'system', 'content': 'You are a helpful assistant.'}
---
{'role': 'user', 'content': 'What is the time and weather in Sydney?'}
---
{'content': "<think>\nOkay, the user is asking for the current time and weather in Sydney. Since I can't access real-time data, I need to let them know that. But I should still be helpful. Maybe suggest checking a reliable website or app for that info. Also, mention that Sydney is in Australia, just in case there's any confusion with other Sydneys. Keep the response friendly and offer further help if needed.\n\nAlright, structure the response: Start by stating I can't provide real-time data. Then suggest using a weather website or app. Mention the time zone part. Offer to help convert time zones if they need. Keep it concise and polite.\n</think>\n\nI can't provide real-time data like current weather or time, but you can quickly check these details for Sydney, Australia, using services like **Google Search**, **Weather.com**, or a weather app. Syd

## Send results of Tool Calls

after the function call is executed, we need to send the result back to the chat completion API.

In [40]:
if finish_reason == CompletionsFinishReason.TOOL_CALLS:
    final_response = client.complete(
        messages=message_history,
        model = selected_model,
        tools=tools,
        tool_choice="auto"
    )

    print("Final Response: ", final_response.choices[0].message.content)
